In [ ]:
import matplotlib.pyplot as plt
import os
from datetime import datetime

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image

import sklearn
from sklearn import metrics
import seaborn as sns
tf.__version__, sklearn.__version__

In [ ]:
SEED = 42
tf.random.set_seed(SEED)

# 1. Setup dataset

In [ ]:
# Setup dataset

%run "./setup_dataset.ipynb"

# 2. Train

### 2.1 modeling

In [ ]:
# define model

def create_model(activation="relu", dropout=0.1, input_shape=(150,150,3), nb_classes=3):
    model = models.Sequential()
    
    model.add(layers.Input(input_shape))
    
    # model.add(layers.Conv2D(32, (3,3), input_shape=input_shape))
    model.add(layers.Conv2D(32, (3,3)))
    model.add(layers.Activation(activation))
    model.add(layers.Dropout(dropout))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3)))
    model.add(layers.Activation(activation))
    model.add(layers.Dropout(dropout))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3)))
    model.add(layers.Activation(activation))
    model.add(layers.Dropout(dropout))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3)))
    model.add(layers.Activation(activation))
    model.add(layers.Dropout(dropout))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation=activation))
    model.add(layers.Dense(nb_classes, activation='softmax'))

    return model

    
model = create_model(activation="relu", dropout=0.3, input_shape=(150,150,3), nb_classes=3)
model.summary()

In [ ]:
# compile model

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False),
              metrics=['accuracy'])
model

In [ ]:
# define generator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 10

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize all images to 150x150.
    batch_size=batch_size,
    class_mode='categorical',
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
)

In [ ]:
# # define generator with data augumentation

# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,)

# test_datagen = ImageDataGenerator(rescale=1./255)

# batch_size = 10

# train_generator = train_datagen.flow_from_directory(
#     train_dir,
#     target_size=(150, 150),  # Resize all images to 150x150.
#     batch_size=batch_size,
#     class_mode='categorical',
# )

# validation_generator = test_datagen.flow_from_directory(
#     validation_dir,
#     target_size=(150, 150),
#     batch_size=batch_size,
#     class_mode='categorical',
# )

In [ ]:
# set early-stopping

best_model = f"best_model_{tf.__version__}.keras"

callbacks = [EarlyStopping(monitor="val_loss", min_delta=1e-5, patience=10, mode="auto", verbose=0),
             ModelCheckpoint(filepath=os.path.join(MODEL_DIR, best_model), monitor="val_loss", save_best_only=True, save_weights_only=False),
             ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=10),  # divide learning_rate by 10 when the callback is invoked.
            ]

In [ ]:
# train

epochs = 30
# epochs = 2

begin_time = datetime.now()

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1,
)

print("\nElapsed Time: {}\t{}".format(datetime.now() - begin_time, datetime.now()))

In [ ]:
# draw convergence history

def draw_convergence_history(history, save_file=None):
    """draw convergence history"""
    
    _, ax = plt.subplots(1,2, figsize=(10,4), facecolor="w")
    
    # obtain the history of training loss and validation loss
    training_loss = history["loss"]
    validation_loss = history["val_loss"]

    # prepare epoch list
    epoch_count = range(1, len(training_loss) + 1)
    
    # plot "loss convergence" for training data and validation data
    ax[0].plot(epoch_count, training_loss, "rx--")  # r-- ro ro- ro--
    ax[0].plot(epoch_count, validation_loss, "bo-")
    
    ax[0].set_title("convergence history of loss")
    ax[0].legend(["Training Loss", "Validation Loss"], loc="upper right")
    ax[0].set_xlabel("Epoch")
    ax[0].set_ylabel("Loss")
    ax[0].grid()

    # plot "accuracy convergence" for training data and validation data
    training_accuracy = history["accuracy"]
    validation_accuracy = history["val_accuracy"]
    
    ax[1].plot(epoch_count, training_accuracy, "rx--")  # r-- ro ro- ro--
    ax[1].plot(epoch_count, validation_accuracy, "bo-")
    
    ax[1].set_title("convergence history of score")
    ax[1].legend(["Training Score", "Validation Score"], loc="lower right")
    ax[1].set_xlabel("Epoch")
    ax[1].set_ylabel("Score")
    ax[1].grid()

    if save_file:
        plt.savefig(save_file)

    plt.show()
    plt.close()

draw_convergence_history(history.history, save_file=Path(OUTPUT_DIR, "convergence_history.png"))

### 2.2 Evaluation

In [ ]:
# define generator

batch_size = 10

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
)

In [ ]:
# calculate loss and accuracy

loss, accuracy = model.evaluate(test_generator, verbose=1)
print(f"{loss= :.4f}\t{accuracy= :.4f}")

In [ ]:
# predict testset

test_pred_dict = {}

for test_dir in test_dirs:
    pred = []
        
    for path in test_dir.iterdir():
#         print(path)
        img = image.load_img(path, target_size=(150, 150))
        img_tensor = image.img_to_array(img)
        img_tensor = np.expand_dims(img_tensor, axis=0)
        img_tensor /= 255.

        p = model.predict(img_tensor)[0]
        pred.append(np.argmax(p))
    
    test_pred_dict[test_dir.name] = pred


test_pred = [*test_pred_dict["salad"], *test_pred_dict["sushi"], *test_pred_dict["tofu"]]
print(test_pred)

In [ ]:
# create ture class list 

num_of_salad = len(os.listdir(test_salad_dir))
num_of_sushi = len(os.listdir(test_sushi_dir))
num_of_tofu = len(os.listdir(test_tofu_dir))

test_true = [*[0]*num_of_salad, *[1]*num_of_sushi, *[2]*num_of_tofu]
print(test_true)

In [ ]:
# classification report

report = metrics.classification_report(test_true, test_pred)

print(f"classification report on test dataset\n{report}")

In [ ]:
# confusion matrix

confusion_matrix = metrics.confusion_matrix(test_true, test_pred)

sns.heatmap(confusion_matrix, annot=True, cmap="Blues", fmt="d")

plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
    
plt.savefig(Path(OUTPUT_DIR, "confusionmatrix.png"))